In [11]:
from google import genai

client = genai.Client(
    vertexai=True,
    project="ignf-simv-inference",
    location="us-central1"
)

# Texte source Wikipédia (exemple, tu peux charger depuis fichier)
wikipedia_text = """
{{Voir homonymes|Rue de Lyon}}
{{ébauche|Paris}}
{{Infobox Voie parisienne
 | num arr        = {{12e}}
 | nom            = Rue de Lyon
 | latitude       = 48.849323
 | longitude      = 2.370788
 | arrondissement = [[12e arrondissement (Paris)|{{12e}}]]
 | quartier       = [[Quartier des Quinze-Vingts|Quinze-Vingts]]
 | début          = [[Place des Combattants-en-Afrique-du-Nord]]
 | fin            = 52, [[boulevard de la Bastille]] et [[place de la Bastille]]
 | longueur       = 710
 | largeur        = 30
 | création       = [[Ordonnance (droit)|Ordonnance]] du {{date|27|novembre|1847}}
 | dénomination   = {{date|27|novembre|1847}}
 | ancien nom     = 
 | photo          = P1160116 Paris XII rue de Lyon rwk.jpg
 | légende        = La rue de Lyon vue depuis le [[boulevard de la Bastille]] ; à gauche, des batiments de l'[[opéra Bastille]] et en arrière-plan, la [[Paris-Gare-de-Lyon|gare de Lyon]].
 | Ville de Paris = 5795
 | DGI            = 5866
 | commons        = Category:Rue de Lyon (Paris)
}}
La '''rue de Lyon''' est située dans le [[quartier des Quinze-Vingts]] du [[12e arrondissement de Paris|{{12e|arrondissement}}]] de [[Paris]].

== Situation et accès ==
Elle débute [[place des Combattants-en-Afrique-du-Nord]], située au carrefour qu'elle forme avec le [[boulevard Diderot]], devant la [[Paris-Gare-de-Lyon|gare de Lyon]] dont elle tire son nom, et se dirige vers la [[place de la Bastille]] en longeant l’[[opéra Bastille]], après avoir croisé notamment l’[[avenue Ledru-Rollin]] et avoir été rejointe par l'[[avenue Daumesnil (Paris)|avenue Daumesnil]].

== Origine du nom ==
Cette rue prend son nom actuel en référence à la [[Paris-Gare-de-Lyon|gare de Lyon]].

== Historique ==
[[File:Paris-Rue de Lyon-1910.jpg|thumb|left|[[Crue de la Seine de 1910]].]]
La ville s'engage auprès de la [[compagnie du chemin de fer de Paris à Lyon]] à faire établir à ses frais les abords de l'[[Paris-Gare-de-Lyon|embarcadère de Lyon]] si celui-ci est établi boulevard Mazas (actuel [[boulevard Diderot]]), dont la création a été ordonnée en 1845.

Le {{date-|27 novembre 1847}}, une ordonnance royale décide donc d'ouvrir une rue entre la [[place de la Bastille]] et la nouvelle gare. L’endroit est peu urbanisé et est occupé, principalement, par des maraîchers<ref>{{Ouvrage|auteur1=André Morizet|titre=Du vieux Paris au Paris moderne|lieu=Paris|éditeur=Librairie Hachette|année=1932|passage=page 144.}}</ref>. Cette rue prend dès l'origine son nom actuel. Sa création entraine la suppression de la rue Treilhard<ref>{{ouvrage|titre=Ville de Paris|sous-titre=recueil des lettres patentes, ordonnances royales, décrets et arrêtés préfectoraux concernant les voies publiques|prénom1=Adolphe|nom1=Alphand|lien auteur1=Jean-Charles Alphand|directeur1=oui|prénom2=Adrien|nom2=Deville|prénom3=Émile|nom3=Hochereau|lieu=Paris|année=1886|éditeur=Imprimerie nouvelle (association ouvrière)|titre chapitre=Ordonnance royale du 27 novembre 1847|passage=232-233}}.</ref>.

[[Bombardements de Paris et de sa banlieue durant la Première Guerre mondiale|Le 13 avril 1918]], durant la [[première Guerre mondiale]], un obus lancé par la [[Pariser Kanonen|Grosse Bertha]] explose au {{Numéro|15}} rue de Lyon<ref>[https://gallica.bnf.fr/ark:/12148/bpt6k4605797h/f6.item.zoom    ''Excelsior du 9 janvier 1919 : Carte et liste officielles des obus lancés par le canon monstre et numérotés suivant leur ordre et leur date de chute'']</ref>.

== Bâtiments remarquables et lieux de mémoire ==
La rue de Lyon est citée dans la chanson ''Ça c'est de la bagnole'', chantée par [[Georgius]]<ref>[http://www.paroles-musique.com/paroles-Georgius-Ca_cest_de_la_bagnole-lyrics,p038646574 Paroles de ''Ça c'est de la bagnole''].</ref>.
* La rue de Lyon longe l'[[opéra Bastille]] et débouche sur la [[place de la Bastille]] et la [[colonne de Juillet]].
* La [[Promenade plantée]] sur le [[viaduc des Arts]].
* {{numéro avec majuscule|12}} : rue de Lyon, cinéma [[UGC Lyon Bastille]], anciennement [[UGC Gare de Lyon]].
* {{numéro avec majuscule|75}} : [[immeuble de l'îlot Biscornet]].

<gallery>
Rue de Lyon (Nord).JPG|La rue de Lyon dans sa section nord avec la [[colonne de Juillet]] au loin.
Rue de Lyon (sud).JPG|La rue dans sa section sud avec l'horloge de la [[Paris-Gare-de-Lyon|gare de Lyon]] dans le lointain.
</gallery>

== Références ==
{{références}}

{{Portail|Paris|route}}

[[Catégorie:Rue dans le 12e arrondissement de Paris|Lyon]]

"""

# Prompt pour le LLM
prompt = """
Tu es un assistant qui extrait des informations clés d'un texte sur une rue de Paris (ici « rue de Lyon »).  
Pour chaque information, génère un objet JSON avec ces règles :

1. L’identifiant `"id"` doit commencer par `11970_`, suivi du type d’information (historique, dénomination, numérotation, ouverture ou classement).  
   - Si un même type apparaît plusieurs fois, ajoute `_2`, `_3`, … pour distinguer (ex : `11970_historique_1`, puis `11970_historique_2`, etc.).  

2. Le champ `"sent"` doit contenir la phrase sous la forme : rue de Lyon || <Type> || <Complément>.


3. N’inclure que ces cinq types :  
- historique  
- dénomination  
- numérotation  
- ouverture  
- classement  

4. Ne retourne **que** la liste JSON d’objets, sans aucun autre texte.

Exemple de sortie attendue :
```json
[
  {{"id": "11970_denomination", "sent": "rue de Lyon || Dénomination || Ordonnance du 27 novembre 1847."}},
  {{"id": "11970_historique_1", "sent": "rue de Lyon || Historique || Crue de la Seine de 1910."}},
  {{"id": "11970_historique_2", "sent": "rue de Lyon || Historique || Bombardement du 13 avril 1918."}}
]

Ne génère que les types suivants : historique, dénomination, numérotation, ouverture, classement.  
Voici le texte :  
{wikipedia_text}


"""

import json

def generate_paraphrases(text, prompt):
    full_prompt = prompt.format(wikipedia_text=text)
    response = client.models.generate_content(
        model="gemini-2.5-pro-preview-05-06",
        contents=full_prompt,
    )
    return response.text

# Appel LLM
json_output = generate_paraphrases(wikipedia_text, prompt)
try:
    data = json.loads(json_output)
    print(data)
except Exception as e:
    print("⚠️ LLM输出不是合法JSON，原始内容如下：")
    print(json_output)

⚠️ LLM输出不是合法JSON，原始内容如下：
```json
[
  {"id": "11970_ouverture_1", "sent": "rue de Lyon || Ouverture || Ordonnance du 27 novembre 1847."},
  {"id": "11970_denomination_1", "sent": "rue de Lyon || Dénomination || Date de dénomination : 27 novembre 1847."},
  {"id": "11970_denomination_2", "sent": "rue de Lyon || Dénomination || Prend son nom actuel en référence à la gare de Lyon."},
  {"id": "11970_historique_1", "sent": "rue de Lyon || Historique || La ville s'engage auprès de la compagnie du chemin de fer de Paris à Lyon à faire établir à ses frais les abords de l'embarcadère de Lyon si celui-ci est établi boulevard Mazas (actuel boulevard Diderot), dont la création a été ordonnée en 1845."},
  {"id": "11970_historique_2", "sent": "rue de Lyon || Historique || Une ordonnance royale du 27 novembre 1847 décide d'ouvrir une rue entre la place de la Bastille et la nouvelle gare."},
  {"id": "11970_historique_3", "sent": "rue de Lyon || Historique || Sa création entraine la suppression de la

In [ ]:
# 设置文件路径
input_file = os.path.normpath(os.path.join(work_dir, "llm_responses/output_llm_responses_part1.jsonl"))
output_cleaned_file = os.path.normpath(os.path.join(work_dir, "llm_responses/cleaned_output_llm_responses_part1.jsonl"))
output_sampled_file = os.path.normpath(os.path.join(work_dir, "llm_responses/sampled_100_output.jsonl"))

# 步骤 1：清洗数据，只保留 response 字段
cleaned_data = []

with open(input_file, 'r', encoding='utf-8') as fin:
    for line in fin:
        obj = json.loads(line)
        # 如果没有 response 但有 sent，就用 sent 替换 response
        response = obj.get("response", obj.get("sent", ""))
        triples = obj.get("triples", [])
        uid = obj.get("id", "")
        cleaned_data.append({"id": uid, "response": response, "triples": triples})

# 保存清洗后的文件
with open(output_cleaned_file, 'w', encoding='utf-8') as fout:
    for item in cleaned_data:
        json.dump(item, fout, ensure_ascii=False)
        fout.write('\n')

# 步骤 2：随机抽取 100 条用于评估（你可以改为别的数字）
sample_size = 100
sampled_data = random.sample(cleaned_data, min(sample_size, len(cleaned_data)))

# 保存抽样后的文件
with open(output_sampled_file, 'w', encoding='utf-8') as fout:
    for item in sampled_data:
        json.dump(item, fout, ensure_ascii=False)
        fout.write('\n')

print(f"✅ 已生成 {output_cleaned_file} 和 {output_sampled_file}")

In [ ]:
# 读取CSV
nomenclature_file = os.path.normpath(os.path.join(work_dir, "nomenclature-denominations-emprises-voies-actuelles.csv"))
df = pd.read_csv(nomenclature_file, dtype=str, sep=';')
print(0/0)

id_col = "Identifiant"
denom_col = "Dénomination complète minuscule"
info_cols = ["Historique", "Dénomination", "Classement", "Ouverture", "Numérotation"]

jsonl_lines = []

for idx, row in df.iterrows():
    exceptions = {
        "dr.":"dr",
        "larg.":"larg",
        "arr. mun.":"arr mun",
        "M.": "M",
        "jan." : "jan",
        "févr.": "févr",
        "avr.": "avr",
        "juil.": "juil",
        "sept.": "sept",
        "oct.": "oct",
        "nov.": "nov",
        "déc.": "déc",
        "Arr.":  "arr",
        "; place Cabanis ;":  ", place Cabanis,",
        ". (ponts)": " (ponts)",
        "Ord.": "Ord",
        "préf.": "préf",
        "ord.": "ord",
        "arr.": "arr",
        "(U. P.)": "U P",
        " ; rue Puteaux": ", rue Puteaux",
        " L'arr.": " L'arr",
        ". mun." : " mun",
        "(sup.)": " (sup)"
      }

    identifiant = str(row.get(id_col, '')).strip()
    denom = str(row.get(denom_col, '')).strip().lower()
    if not identifiant or not denom:
        continue  # 跳过无效行

    for col in info_cols:
        info = str(row.get(col, '')).strip()
        if not info or info.lower() in ['nan', 'none']:
            continue  # 跳过空内容

        # # 多条信息分割
        # if ';' in info:
        #     infos = [i.strip() for i in info.split(';') if i.strip()]
        # elif '\n' in info:
        #     infos = [i.strip() for i in info.split('\n') if i.strip()]
        # else:
        #     infos = [info]

        # 多条信息分割
        
        for abbr, full in exceptions.items():
            info = info.replace(abbr, full)

        # On découpe selon le point ".", le point virgule ";" ou le saut de ligne "\n"
        infos = re.split(r'[;.\n]', info)

        # On enlève les espaces vides et les chaînes vides
        infos = [x.strip() for x in infos if x.strip()]

        for i, single_info in enumerate(infos):
            if len(infos) == 1:
                id_str = f"{identifiant}_{col.lower()}"
            else:
                id_str = f"{identifiant}_{col.lower()}_{i+1}"
            sent = f"{denom} || {col} || {single_info}"
            jsonl_lines.append(json.dumps({"id": id_str, "sent": sent}, ensure_ascii=False))

# 写入jsonl文件
with open('output.jsonl', 'w', encoding='utf-8') as f:
    for line in jsonl_lines:
        f.write(line + '\n')

print(f"已生成 {len(jsonl_lines)} 条数据，保存在 output.jsonl")